In [ ]:
from numpy import set_printoptions
import pandas as pd
import os
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from rdkit import Chem
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression                         
from sklearn.tree import DecisionTreeClassifier                             
from sklearn.neighbors import KNeighborsClassifier                          
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis        
from sklearn.naive_bayes import GaussianNB                                  
from sklearn.svm import SVC    
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_rdkit = pd.read_csv("./BBBP_rdkit_descriptors.csv") 
print(df_rdkit.head())

In [ ]:
description = df_rdkit.describe()
print(description)

In [ ]:
df_cleaned = df_rdkit.drop(columns=['name'], errors='ignore')


In [ ]:
#correlation-based feature reduction
corr = df_cleaned.corr()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
df_uncorrelated = df_cleaned.drop(columns=to_drop)

In [ ]:
# Removing features with low variance: 
X_all =  df_uncorrelated.drop(columns=['p_np'])
Y_all =  df_uncorrelated['p_np']
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X_all)

In [ ]:
###  Univariate feature selection: this selects features most strongly related to the target (based on F-score).

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd

# Select top k features using ANOVA F-test
k = 30  # or any number of top features you want
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X_all, Y_all)

# Get selected feature names
mask = selector.get_support()  # boolean mask of selected features
selected_features = X_all.columns[mask]

# Show selected features and their F-scores
scores = selector.scores_[mask]
feature_scores = pd.Series(scores, index=selected_features).sort_values(ascending=False)
print(" Top features selected using ANOVA F-test:\n")
print(feature_scores)

In [ ]:

selected_features_30 = [
    "TPSA", "NHOHCount", "PEOE_VSA10", "PEOE_VSA1", "qed", "fr_lactam", "SMR_VSA2", "EState_VSA8", "SlogP_VSA2",
    "VSA_EState2", "NumRotatableBonds", "MolWt", "SlogP_VSA3", "PEOE_VSA13", "fr_sulfide", "fr_Al_COO",
    "VSA_EState3", "MolLogP", "Phi", "PEOE_VSA12", "NumAtomStereoCenters", "EState_VSA10", "fr_Al_OH",
    "HallKierAlpha", "fr_Ar_OH", "PEOE_VSA9", "PEOE_VSA5", "NumSaturatedHeterocycles", "SMR_VSA10", "PEOE_VSA2"
]

# Create a new dataset with just these features + target
selected_df = df_cleaned[selected_features_30 + ['p_np']].copy()



In [ ]:

array = selected_df.values
X = selected_df.drop(columns=['p_np'])
y = selected_df['p_np']

corr_matrix = X.corr()

In [ ]:
plt.figure(figsize=(30, 20))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True, linewidths=0.5)
plt.title("Correlation Matrix of Top 20 Features")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:

X_all =  df_uncorrelated.drop(columns=['p_np'])
Y_all =  df_uncorrelated['p_np']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_all)

 BENCHMARK COMPARISON\
 Benchmark comparison of multiple machine learning models using cross-validation, and evaluating them based on ROC-AUC and F1-score.

In [ ]:

# prepare models
models = []
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=500)))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(probability=True)))  # Important: probability=True for SVC

# evaluate each model
roc_results = []
f1_results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=10, random_state=7, shuffle=True)

    # ROC-AUC
    cv_roc = cross_val_score(model, X_scaled, Y_all, cv=kfold, scoring='roc_auc')
    roc_results.append(cv_roc)

    # F1-Score
    cv_f1 = cross_val_score(model, X_scaled, Y_all, cv=kfold, scoring='f1')
    f1_results.append(cv_f1)

    names.append(name)
    
    print(f"{name}:")
    print(f"  ROC-AUC Mean: {cv_roc.mean():.3f} (Std: {cv_roc.std():.3f})")
    print(f"  F1-Score Mean: {cv_f1.mean():.3f} (Std: {cv_f1.std():.3f})\n")

In [ ]:
# boxplot algorithm comparison
fig = pyplot.figure()
fig.suptitle('Algorithms comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(roc_results)
ax.set_xticklabels(names)
pyplot.show()

In [ ]:
fig = plt.figure(figsize=(10,6))
fig.suptitle('Algorithms Comparison - F1 Scores')
ax = fig.add_subplot(111)
plt.boxplot(f1_results)
ax.set_xticklabels(names)
plt.ylabel('F1 Score')
plt.grid()
plt.show()

IMBALANCED CLASSIFICATION\
Imbalanced classification strategy comparison for logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(
    X_scaled, Y_all, test_size=0.2, stratify=Y_all, random_state=42
)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# ----- 1. Logistic Regression with class_weight=none -----
# On imbalanced datasets, it tends to favor the majority class.

model_none = LogisticRegression(solver='lbfgs', max_iter=300, class_weight=None)

y_pred_none= cross_val_predict(model_none, X_scaled, Y_all, cv=cv)
y_prob_none = cross_val_predict(model_none, X_scaled, Y_all, cv=cv, method='predict_proba')[:, 1]

acc_n = accuracy_score(Y_all, y_pred_none)
roc_n = roc_auc_score(Y_all, y_prob_none)
print(" LogisticRegression with class_weight='balanced'")
print("Accuracy:", round(acc_n, 3))
print("ROC-AUC:", round(roc_n, 3))
print("Classification Report:\n", classification_report(Y_all, y_pred_none))

# ----- 2. Logistic Regression with class_weight='balanced' -----
#  The “balanced” mode uses the values of y to automatically adjust weights 
#  inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)).


model_weighted = LogisticRegression(solver='lbfgs', max_iter=300, class_weight='balanced')

y_pred_weighted = cross_val_predict(model_weighted, X_scaled, Y_all, cv=cv)
y_prob_weighted = cross_val_predict(model_weighted, X_scaled, Y_all, cv=cv, method='predict_proba')[:, 1]

acc_w = accuracy_score(Y_all, y_pred_weighted)
roc_w = roc_auc_score(Y_all, y_prob_weighted)
print(" LogisticRegression with class_weight='balanced'")
print("Accuracy:", round(acc_w, 3))
print("ROC-AUC:", round(roc_w, 3))
print("Classification Report:\n", classification_report(Y_all, y_pred_weighted))


# ----- 3. SMOTE + Logistic Regression -----
# Applies SMOTE within cross-validation folds to generate synthetic minority samples: DOI: https://doi.org/10.1613/jair.953

smote = SMOTE(random_state=42)
model_smote = LogisticRegression(solver='lbfgs', max_iter=300)

pipeline = Pipeline([
    ('smote', smote),
    ('logreg', model_smote)
])

y_pred_smote = cross_val_predict(pipeline, X_scaled, Y_all, cv=cv)
y_prob_smote = cross_val_predict(pipeline, X_scaled, Y_all, cv=cv, method='predict_proba')[:, 1]

acc_s = accuracy_score(Y_all, y_pred_smote)
roc_s = roc_auc_score(Y_all, y_prob_smote)
print("\n LogisticRegression with SMOTE")
print("Accuracy:", round(acc_s, 3))
print("ROC-AUC:", round(roc_s, 3))
print("Classification Report:\n", classification_report(Y_all, y_pred_smote))


# ----- Confusion Matrices -----

fig, axs = plt.subplots(1, 3, figsize=(18, 4))  

cm_none = confusion_matrix(Y_all, y_pred_none)
sns.heatmap(cm_none, annot=True, fmt='d', cmap='Reds', ax=axs[0])
axs[0].set_title("Confusion Matrix\nclass_weight=None")
axs[0].set_xlabel("Predicted")
axs[0].set_ylabel("Actual")


cm_weighted = confusion_matrix(Y_all, y_pred_weighted)
sns.heatmap(cm_weighted, annot=True, fmt='d', cmap='Blues', ax=axs[1])
axs[1].set_title("Confusion Matrix\nclass_weight='balanced'")
axs[1].set_xlabel("Predicted")
axs[1].set_ylabel("Actual")

cm_smote = confusion_matrix(Y_all, y_pred_smote)
sns.heatmap(cm_smote, annot=True, fmt='d', cmap='Greens', ax=axs[2])
axs[2].set_title("Confusion Matrix\nSMOTE + LogisticRegression")
axs[2].set_xlabel("Predicted")
axs[2].set_ylabel("Actual")

plt.tight_layout()
plt.show()


From the plot we can see clearly that without balancing we have:\
High FN: model is misclassifying many BBB− as BBB+ \
Low TN:  model rarely predicts class 0 correctly \
High TP:  model may be blindly favoring class 1\

Using class_weight='balanced\
Forces the model to pay attention to both classes, even when one is smaller.

Using SMOTE:\
Creates synthetic examples of the minority class (class 0) during training. Gives the model more balanced data to learn from in each fold.

Unbalanced model biases heavily toward class 1 and ignores class 0. had high recall for class 1 but terrible performance on class 0 (missed many non-penetrants).\
Balanced and SMOTE models give more useful, realistic results — with much better treatment of non-penetrant compounds (class 0).\
Balanced and SMOTE:\
Correctly identified more class 0 compounds.
Gave lower but more reliable accuracy.
Produced better confusion matrices (more TNs, fewer FNs for class 0).


Compares 3 versions of SVM on BBBP:
Untuned SVM (default params)

SVM tuned with GridSearchCV

SVM tuned with Genetic Algorithm

And for each version:

Trains the model

Evaluates with classification_report and ROC AUC

Records and plots training time and ROC AUC



In [ ]:

import pandas as pd
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_selection import VarianceThreshold



# ===========================
#  Comparison Configuration
# ===========================
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['rbf']
}

# Track results
results = {}

# ===============================
# 1. Untuned SVM
# ===============================
start = time.time()
svm_untuned = SVC(probability=True, random_state=42)
svm_untuned.fit(X_train_scaled, y_train)
y_pred = svm_untuned.predict(X_test_scaled)
y_prob = svm_untuned.predict_proba(X_test_scaled)[:, 1]
auc = roc_auc_score(y_test, y_prob)
end = time.time()

results["Untuned"] = {
    "auc": auc,
    "time": end - start,
    "report": classification_report(y_test, y_pred)
}
print("\n Untuned SVM")
print(results["Untuned"]["report"])
print("ROC AUC:", auc)

# ===============================
# 2. GridSearchCV
# ===============================
start = time.time()
grid = GridSearchCV(SVC(probability=True, random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train_scaled, y_train)
best_svm_grid = grid.best_estimator_
y_pred = best_svm_grid.predict(X_test_scaled)
y_prob = best_svm_grid.predict_proba(X_test_scaled)[:, 1]
auc = roc_auc_score(y_test, y_prob)
end = time.time()

results["GridSearchCV"] = {
    "auc": auc,
    "time": end - start,
    "report": classification_report(y_test, y_pred),
    "best_params": grid.best_params_
}
print("\n SVM with GridSearchCV")
print(results["GridSearchCV"]["report"])
print("ROC AUC:", auc)
print("Best Params:", grid.best_params_)

# ===============================
# 3. Genetic Algorithm
# ===============================
def generate_population(size):
    random.seed(42)

    return [
        {
            'C': random.choice(param_grid['C']),
            'gamma': random.choice(param_grid['gamma']),
            'kernel': 'rbf'
        }
        for _ in range(size)
    ]

def fitness(ind):
    model = SVC(**ind, probability=True, random_state=42)
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='roc_auc')
    return scores.mean()
def crossover(p1, p2):
    return {
        'C': random.choice([p1['C'], p2['C']]),
        'gamma': random.choice([p1['gamma'], p2['gamma']]),
        'kernel': 'rbf'
    }

def mutate(ind, mutation_rate=0.1):
    if random.random() < mutation_rate:
        ind['C'] = random.choice(param_grid['C'])
    if random.random() < mutation_rate:
        ind['gamma'] = random.choice(param_grid['gamma'])
    return ind

def run_genetic_algorithm(pop_size=10, generations=5):
    population = generate_population(pop_size)
    best_scores = []

    for gen in range(generations):
        scored = [(ind, fitness(ind)) for ind in population]
        scored.sort(key=lambda x: x[1], reverse=True)
        elites = [x[0] for x in scored[:pop_size // 2]]
        best_scores.append(scored[0][1])
        offspring = [mutate(crossover(*random.sample(elites, 2))) for _ in range(pop_size // 2)]
        population = elites + offspring

    best_params = max([(ind, fitness(ind)) for ind in population], key=lambda x: x[1])[0]



    return best_params

start = time.time()
best_params_ga = run_genetic_algorithm()
svm_ga = SVC(**best_params_ga, probability=True, random_state=42)
svm_ga.fit(X_train_scaled, y_train)
y_pred = svm_ga.predict(X_test_scaled)
y_prob = svm_ga.predict_proba(X_test_scaled)[:, 1]
auc = roc_auc_score(y_test, y_prob)
end = time.time()

results["Genetic Algorithm"] = {
    "auc": auc,
    "time": end - start,
    "report": classification_report(y_test, y_pred),
    "best_params": best_params_ga
}
print("\n SVM with Genetic Algorithm")
print(results["Genetic Algorithm"]["report"])
print("ROC AUC:", auc)
print("Best Params:", best_params_ga)

# ===============================
# Final Comparison Plot
# ===============================
labels = list(results.keys())
auc_scores = [results[k]["auc"] for k in labels]
times = [results[k]["time"] for k in labels]

fig, ax1 = plt.subplots(figsize=(8, 5))

ax1.bar(labels, auc_scores, color='skyblue', label='ROC AUC')
ax1.set_ylabel('ROC AUC')
ax1.set_ylim(0.5, 1.0)

ax2 = ax1.twinx()
ax2.plot(labels, times, color='red', marker='o', label='Time (s)')
ax2.set_ylabel('Time (s)', color='red')

plt.title("SVM Comparison: Untuned vs GridSearchCV vs GA")
fig.tight_layout()
plt.show()


The ROC AUC scores for all three models are very close, around 0.93 to 0.94.\
This means all three models—Untuned SVM, GridSearchCV, and Genetic Algorithm (GA)—perform similarly well in distinguishing between the two classes.\
The small differences in AUC are visually hard to distinguish due to the narrow range (0.93–0.94), but numerically they may still matter depending on the application\
Time increases dramatically from Untuned (~2s) to GridSearchCV (~4s) and jumps significantly for GA (~78s).
